In [13]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------  12.6/12.8 MB 87.4 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 72.9 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Class Text-Vision Comparison - KonkLab

This notebook compares CVCL and CLIP models using text-based prototypes for class discrimination.
Instead of averaging image features to create prototypes, we use text descriptions.
The task remains 4-way forced choice classification with 4000 trials.

In [14]:
import os
import sys
import random
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from collections import defaultdict
import clip

# Path setup - Use absolute paths to avoid any confusion
REPO_ROOT = r'C:\Users\jbats\Projects\NTU-Synthetic'

# Add discover-hidden-visual-concepts to path
DISCOVER_ROOT = os.path.join(REPO_ROOT, 'discover-hidden-visual-concepts')
sys.path.insert(0, DISCOVER_ROOT)
sys.path.insert(0, REPO_ROOT)

# Import from discover-hidden-visual-concepts repo
sys.path.append(os.path.join(DISCOVER_ROOT, 'src'))
from utils.model_loader import load_model
from models.feature_extractor import FeatureExtractor

# SyntheticKonkle paths - Updated from KonkLab
CSV_PATH = os.path.join(REPO_ROOT, 'data', 'SyntheticKonkle', 'master_labels.csv')
# Fixed path - the resized images have an extra nested folder
IMG_DIR = os.path.join(REPO_ROOT, 'data', 'SyntheticKonkle_224', 'SyntheticKonkle')
MASTER_CSV = os.path.join(REPO_ROOT, 'PatrickProject', 'Chart_Generation', 'text_vision_results.csv')

In [15]:
# Quick test to check if GPU is available and model loading speed
import torch
import time

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    
    # Test GPU speed
    x = torch.randn(32, 3, 224, 224).cuda()
    torch.cuda.synchronize()
    start = time.time()
    for _ in range(10):
        _ = x * 2
    torch.cuda.synchronize()
    print(f"GPU test time: {time.time() - start:.3f}s")
else:
    print("WARNING: Running on CPU will be VERY slow!")
    print("If you have a GPU, make sure CUDA is properly installed")

PyTorch version: 2.3.1
CUDA available: True
CUDA device: NVIDIA GeForce RTX 4070 SUPER
CUDA version: 11.8
GPU test time: 0.017s


In [16]:
# Dataset class with optimized loading for SyntheticKonkle
class ClassImageDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform):
        print(f"[DEBUG] Loading CSV from: {csv_path}")
        print(f"[DEBUG] Image directory: {img_dir}")
        
        self.df = pd.read_csv(csv_path)
        print(f"[DEBUG] CSV loaded with {len(self.df)} rows")
        
        # SyntheticKonkle has lowercase 'class' instead of 'Class'
        assert 'filename' in self.df and 'class' in self.df and 'folder' in self.df, \
            "CSV needs filename, class, and folder columns"
        self.img_dir = img_dir
        self.transform = transform
        
        # Pre-compute paths to avoid repeated joins
        # SyntheticKonkle structure: {img_dir}/{folder}/{filename}
        self.paths = [os.path.join(img_dir, row['folder'], row['filename']) 
                      for _, row in self.df.iterrows()]
        
        print(f"[DEBUG] Generated {len(self.paths)} image paths")
        print(f"[DEBUG] First path: {self.paths[0] if self.paths else 'None'}")
        print(f"[DEBUG] First path exists: {os.path.exists(self.paths[0]) if self.paths else 'N/A'}")
        
        # Pre-filter to only valid images
        valid_indices = []
        for idx, path in enumerate(self.paths):
            try:
                # Quick check if image can be opened
                img = Image.open(path)
                img.close()
                valid_indices.append(idx)
            except:
                if idx < 5:  # Print first few failures
                    print(f"[DEBUG] Failed to open image {idx}: {path}")
                continue
        
        print(f"[DEBUG] Found {len(valid_indices)} valid images out of {len(self.paths)}")
        
        # Filter dataframe and paths to only valid images
        self.df = self.df.iloc[valid_indices].reset_index(drop=True)
        self.paths = [self.paths[i] for i in valid_indices]
        print(f"Dataset initialized with {len(self.paths)} valid images")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        cls = row['class']  # lowercase in SyntheticKonkle
        # Use pre-computed path
        path = self.paths[idx]
        try:
            img = Image.open(path).convert('RGB')
        except Exception as e:
            print(f"[ERROR] Failed to load image at index {idx}: {path}")
            # Return a blank image if corrupted
            img = Image.new('RGB', (224, 224), color='white')
        return self.transform(img), cls, idx

def collate_fn(batch):
    imgs = torch.stack([b[0] for b in batch])
    classes = [b[1] for b in batch]
    idxs = [b[2] for b in batch]
    return imgs, classes, idxs

def run_class_text_vision_test(model_name, seed=0, device=None, batch_size=16, 
                                trials_per_class=None, max_trials=4000):
    """
    Run 4-way classification test using the EXACT methodology from discover-hidden-visual-concepts predict() method.
    Now using SyntheticKonkle dataset.
    """
    random.seed(seed)
    torch.manual_seed(seed)
    
    # Device selection
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    if device == 'cuda' and not torch.cuda.is_available():
        print("[ERROR] CUDA requested but not available! Falling back to CPU.")
        device = 'cpu'
    
    if device == 'cpu':
        print("[WARNING] Running on CPU - this will be SLOW!")
        print("Reducing batch size to 4 for CPU")
        batch_size = 4
    else:
        print(f"[INFO] Using GPU: {torch.cuda.get_device_name(0)}")

    # Check if model supports text encoding
    if model_name in ['resnext', 'dino_s_resnext50']:
        print(f"[WARNING] {model_name} has no text encoder, skipping")
        return {}, 0.0

    # 1) Load model
    print(f"[INFO] Loading {model_name} on {device}...")
    import time
    start_time = time.time()
    model, transform = load_model(model_name, seed=seed, device=device)
    extractor = FeatureExtractor(model_name, model, device)
    print(f"[INFO] Model loaded in {time.time() - start_time:.1f}s")

    # 2) Load dataset
    print(f"[DEBUG] Creating dataset with CSV_PATH={CSV_PATH}")
    print(f"[DEBUG] Creating dataset with IMG_DIR={IMG_DIR}")
    ds = ClassImageDataset(CSV_PATH, IMG_DIR, transform)
    print(f"[INFO] Dataset: {len(ds)} images")
    
    if len(ds) == 0:
        print("[ERROR] Dataset is empty! No valid images found.")
        return {}, 0.0
    
    # Use multiple workers only if not on Windows or if on Linux/Mac
    num_workers = 0 if os.name == 'nt' else 2
    
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False,
                   num_workers=num_workers, collate_fn=collate_fn, 
                   pin_memory=(device=='cuda'))
    
    # 3) Extract image embeddings
    print(f"[INFO] Extracting embeddings (batch_size={batch_size})...")
    all_img_embs, all_classes, all_idxs = [], [], []
    
    from tqdm import tqdm
    start_time = time.time()
    
    with torch.no_grad():
        for imgs, classes, idxs in tqdm(dl, desc="Extracting embeddings"):
            imgs = imgs.to(device, non_blocking=True)
            feats = extractor.get_img_feature(imgs)
            feats = extractor.norm_features(feats).cpu()
            
            all_img_embs.append(feats)
            all_classes.extend(classes)
            all_idxs.extend(idxs)
    
    if not all_img_embs:
        print("[ERROR] No embeddings extracted!")
        return {}, 0.0
                
    all_img_embs = torch.cat(all_img_embs, dim=0)
    print(f"[INFO] Extracted {len(all_idxs)} embeddings in {time.time() - start_time:.1f}s")

    # 4) Encode text labels
    unique_classes = list(set(all_classes))
    print(f"[INFO] Encoding {len(unique_classes)} class labels...")
    
    class_text_features = {}
    with torch.no_grad():
        if "clip" in model_name:
            tokens = clip.tokenize(unique_classes, truncate=True).to(device)
            txt_features = model.encode_text(tokens)
            txt_features = extractor.norm_features(txt_features).cpu()
            for i, cls in enumerate(unique_classes):
                class_text_features[cls] = txt_features[i]
        else:  # CVCL
            tokens, token_len = model.tokenize(unique_classes)
            tokens = tokens.to(device)
            if isinstance(token_len, torch.Tensor):
                token_len = token_len.to(device)
            txt_features = model.encode_text(tokens, token_len)
            txt_features = extractor.norm_features(txt_features).cpu()
            for i, cls in enumerate(unique_classes):
                class_text_features[cls] = txt_features[i]
    
    print(f"[INFO] Text encoding complete")
    
    # 5) Build mappings
    idx2class = {i:c for i,c in zip(all_idxs, all_classes)}
    idx2row = {i:r for r,i in enumerate(all_idxs)}
    class2idxs = defaultdict(list)
    for i,c in idx2class.items():
        class2idxs[c].append(i)

    # 6) Run trials - FIXED to run exactly max_trials
    class_results = {}
    total_correct = 0
    total_trials = 0
    
    # Get valid classes (those with at least 1 image)
    valid_classes = [c for c in class2idxs if len(class2idxs[c]) >= 1]
    n_classes = len(valid_classes)
    
    # Calculate trials per class if not specified
    if trials_per_class is None:
        # Distribute trials evenly across classes, with some getting extra
        trials_per_class = max_trials // n_classes
        extra_trials = max_trials % n_classes
    else:
        extra_trials = 0
    
    print(f"[INFO] Running {max_trials} total trials across {n_classes} classes")
    print(f"[INFO] Base trials per class: {trials_per_class}, extra trials for first {extra_trials} classes")
    
    # Run trials for each class
    for class_idx, cls in enumerate(valid_classes):
        if total_trials >= max_trials:
            break
            
        idxs = class2idxs[cls]
        
        # Add extra trial for first few classes to reach exactly max_trials
        current_trials = trials_per_class + (1 if class_idx < extra_trials else 0)
        # Don't exceed max_trials
        current_trials = min(current_trials, max_trials - total_trials)
        
        correct = 0
        txt_feature = class_text_features[cls].unsqueeze(0)
        
        for trial in range(current_trials):
            # Pick query and distractors
            q = random.choice(idxs)
            others = [i for i in all_idxs if idx2class[i] != cls]
            if len(others) < 3:
                continue
            distractors = random.sample(others, 3)
            
            # 4-way classification
            candidates = [q] + distractors
            cand_features = torch.stack([all_img_embs[idx2row[i]] for i in candidates])
            cand_features = cand_features.unsqueeze(0)
            
            # Compute similarity
            txt_feature_expanded = txt_feature.unsqueeze(1)
            similarity = (100.0 * cand_features @ txt_feature_expanded.transpose(-2, -1)).softmax(dim=1)
            similarity = similarity.squeeze()
            
            # Predict (query is at index 0)
            if similarity.argmax().item() == 0:
                correct += 1
                total_correct += 1
            total_trials += 1

        acc = correct / current_trials if current_trials > 0 else 0
        class_results[cls] = {'correct': correct, 'trials': current_trials, 'accuracy': acc}
        
        if total_trials % 500 == 0:
            print(f"[PROGRESS] {total_trials}/{max_trials} trials completed")

    # Final progress update
    print(f"[FINAL] Completed {total_trials} trials")
    
    overall_acc = total_correct / total_trials if total_trials else 0.0
    
    print(f"\n[RESULTS] Accuracy: {total_correct}/{total_trials} ({overall_acc:.1%})")
    
    # Save results
    summary_df = pd.DataFrame([{
        'Model': model_name,
        'Test': 'Class-TextVision-Original',
        'Dataset': 'SyntheticKonkle',  # Changed from KonkLab
        'Correct': total_correct,
        'Trials': total_trials,
        'Accuracy': overall_acc
    }])
    
    os.makedirs(os.path.dirname(MASTER_CSV), exist_ok=True)
    if os.path.exists(MASTER_CSV):
        summary_df.to_csv(MASTER_CSV, mode='a', header=False, index=False, float_format='%.4f')
    else:
        summary_df.to_csv(MASTER_CSV, index=False, float_format='%.4f')

    return class_results, overall_acc

## CVCL Text-Vision Test

In [17]:
# Run CVCL text-vision classification
cvcl_results, cvcl_overall = run_class_text_vision_test('cvcl-resnext', max_trials=4000)

print("\nCVCL Results per Class:")
for cls, res in sorted(cvcl_results.items(), key=lambda x: x[1]['accuracy'], reverse=True)[:20]:
    print(f"{cls:20s}: {res['correct']}/{res['trials']} ({res['accuracy']:.1%})")
print(f"\nCVCL Overall Accuracy: {cvcl_overall:.1%}")

[INFO] Using GPU: NVIDIA GeForce RTX 4070 SUPER
[INFO] Loading cvcl-resnext on cuda...
Loading checkpoint from C:\Users\jbats\.cache\huggingface\hub\models--wkvong--cvcl_s_dino_resnext50_embedding\snapshots\f50eaa0c50a6076a5190b1dd52aeeb6c3e747045\cvcl_s_dino_resnext50_embedding.ckpt


Lightning automatically upgraded your loaded checkpoint from v1.5.8 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\jbats\.cache\huggingface\hub\models--wkvong--cvcl_s_dino_resnext50_embedding\snapshots\f50eaa0c50a6076a5190b1dd52aeeb6c3e747045\cvcl_s_dino_resnext50_embedding.ckpt`


[INFO] Model loaded in 0.6s
[DEBUG] Creating dataset with CSV_PATH=C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\master_labels.csv
[DEBUG] Creating dataset with IMG_DIR=C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle_224\SyntheticKonkle
[DEBUG] Loading CSV from: C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\master_labels.csv
[DEBUG] Image directory: C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle_224\SyntheticKonkle
[DEBUG] CSV loaded with 7882 rows
[DEBUG] Generated 7882 image paths
[DEBUG] First path: C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle_224\SyntheticKonkle\abacus_color\abacus_large_bumpy_01_red.png
[DEBUG] First path exists: True
[DEBUG] Found 7849 valid images out of 7882
Dataset initialized with 7849 valid images
[INFO] Dataset: 7849 images
[INFO] Extracting embeddings (batch_size=16)...


Extracting embeddings: 100%|██████████| 491/491 [00:22<00:00, 22.28it/s]


[INFO] Extracted 7849 embeddings in 22.0s
[INFO] Encoding 67 class labels...
[INFO] Text encoding complete
[INFO] Running 4000 total trials across 67 classes
[INFO] Base trials per class: 59, extra trials for first 47 classes
[PROGRESS] 1500/4000 trials completed
[PROGRESS] 4000/4000 trials completed
[FINAL] Completed 4000 trials

[RESULTS] Accuracy: 1086/4000 (27.2%)

CVCL Results per Class:
ring                : 44/60 (73.3%)
grill               : 38/60 (63.3%)
butterfly           : 31/60 (51.7%)
microwave           : 31/60 (51.7%)
apple               : 30/60 (50.0%)
phone               : 30/60 (50.0%)
stapler             : 29/59 (49.2%)
basket              : 29/60 (48.3%)
wineglass           : 23/59 (39.0%)
bonzai              : 23/60 (38.3%)
dresser             : 23/60 (38.3%)
telescope           : 21/59 (35.6%)
goggle              : 21/60 (35.0%)
doll                : 20/60 (33.3%)
pumpkin             : 20/60 (33.3%)
seashell            : 19/59 (32.2%)
horse               : 19/59 

## CLIP Text-Vision Test

In [18]:
# Run CLIP text-vision classification
clip_results, clip_overall = run_class_text_vision_test('clip-resnext', max_trials=4000)

print("\nCLIP Results per Class:")
for cls, res in sorted(clip_results.items(), key=lambda x: x[1]['accuracy'], reverse=True)[:20]:
    print(f"{cls:20s}: {res['correct']}/{res['trials']} ({res['accuracy']:.1%})")
print(f"\nCLIP Overall Accuracy: {clip_overall:.1%}")

[INFO] Using GPU: NVIDIA GeForce RTX 4070 SUPER
[INFO] Loading clip-resnext on cuda...
[INFO] Model loaded in 1.7s
[DEBUG] Creating dataset with CSV_PATH=C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\master_labels.csv
[DEBUG] Creating dataset with IMG_DIR=C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle_224\SyntheticKonkle
[DEBUG] Loading CSV from: C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle\master_labels.csv
[DEBUG] Image directory: C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle_224\SyntheticKonkle
[DEBUG] CSV loaded with 7882 rows
[DEBUG] Generated 7882 image paths
[DEBUG] First path: C:\Users\jbats\Projects\NTU-Synthetic\data\SyntheticKonkle_224\SyntheticKonkle\abacus_color\abacus_large_bumpy_01_red.png
[DEBUG] First path exists: True
[DEBUG] Found 7849 valid images out of 7882
Dataset initialized with 7849 valid images
[INFO] Dataset: 7849 images
[INFO] Extracting embeddings (batch_size=16)...


Extracting embeddings:   0%|          | 0/491 [00:00<?, ?it/s]c:\Users\jbats\miniconda3\envs\ntu-synthetic\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Extracting embeddings: 100%|██████████| 491/491 [00:18<00:00, 26.10it/s]


[INFO] Extracted 7849 embeddings in 18.8s
[INFO] Encoding 67 class labels...
[INFO] Text encoding complete
[INFO] Running 4000 total trials across 67 classes
[INFO] Base trials per class: 59, extra trials for first 47 classes
[PROGRESS] 1500/4000 trials completed
[PROGRESS] 4000/4000 trials completed
[FINAL] Completed 4000 trials

[RESULTS] Accuracy: 3367/4000 (84.2%)

CLIP Results per Class:
bird                : 60/60 (100.0%)
calculator          : 60/60 (100.0%)
doll                : 60/60 (100.0%)
earrings            : 60/60 (100.0%)
muffin              : 60/60 (100.0%)
tennisracquet       : 59/59 (100.0%)
rabbit              : 59/59 (100.0%)
wineglass           : 59/59 (100.0%)
apple               : 59/60 (98.3%)
axe                 : 59/60 (98.3%)
christmastreeornamentball: 59/60 (98.3%)
handbag             : 59/60 (98.3%)
handgun             : 59/60 (98.3%)
horse               : 58/59 (98.3%)
vase                : 58/59 (98.3%)
nailpolish          : 58/60 (96.7%)
pen            

## Comparison Summary

In [19]:
print("="*60)
print("TEXT-VISION vs VISUAL PROTOTYPE COMPARISON")
print("="*60)
print(f"\nText-Vision Classification (using text descriptions as prototypes):")
print(f"  CVCL: {cvcl_overall:.1%}")
print(f"  CLIP: {clip_overall:.1%}")
print(f"  CLIP advantage: {(clip_overall - cvcl_overall)*100:.1f} percentage points")
print(f"\nNote: Compare these results with the visual prototype version")
print(f"to see how text-based prototypes perform vs image-based prototypes.")

TEXT-VISION vs VISUAL PROTOTYPE COMPARISON

Text-Vision Classification (using text descriptions as prototypes):
  CVCL: 27.2%
  CLIP: 84.2%
  CLIP advantage: 57.0 percentage points

Note: Compare these results with the visual prototype version
to see how text-based prototypes perform vs image-based prototypes.
